In [2]:
# ============================================
# 저장된 VotingClassifier (.pkl) 불러오기 + 예측
# ============================================

import joblib
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

# -------------------------------
# 설정
# -------------------------------
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
MAX_LEN = 256
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[Device] {device}")

# -------------------------------
# 1) 저장된 pkl 불러오기
# -------------------------------
SAVE_PKL = "./models.pkl"
data = joblib.load(SAVE_PKL)

clf = data["classifier"]
mlb = data["mlb"]
thresholds = data["thresholds"]

print(f"[Loaded model from {SAVE_PKL}]")
print(f"Labels: {list(mlb.classes_)}")

# -------------------------------
# 2) MiniLM 로드 (임베딩 추출용)
# -------------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModel.from_pretrained(MODEL_NAME).to(device)
base_model.eval()

def encode_texts(texts, batch_size=32):
    """텍스트를 MiniLM 임베딩으로 변환"""
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt").to(device)
        with torch.no_grad():
            model_out = base_model(**enc)
            emb = model_out.last_hidden_state.mean(dim=1)
        all_embeddings.append(emb.cpu().numpy())
    return np.vstack(all_embeddings)

# -------------------------------
# 3) 예측 함수
# -------------------------------
def predict_multilingual(text: str, topk=3, thresholds=None):
    emb = encode_texts([text], batch_size=1)
    proba = clf.predict_proba(emb)[0]

    if thresholds is not None:
        pick = [i for i, p in enumerate(proba) if p >= thresholds.get(mlb.classes_[i], 0.5)]
        if not pick:  # 어떤 것도 threshold 못 넘으면 topk 선택
            pick = np.argsort(-proba)[:topk]
    else:
        pick = np.argsort(-proba)[:topk]

    return [mlb.classes_[i] for i in pick]


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Device] cpu


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [13:59:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:359: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [13:59:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:384: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [13:59:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  setstate(state)
c:\Users\Playdata2

[Loaded model from ./models.pkl]
Labels: ['Amber', 'Aromatic', 'Blossom', 'Bouquet', 'Citrus', 'Classical', 'Crisp', 'Dry', 'Floral', 'Flower', 'Fougère', 'Fresh', 'Fresher', 'Fruity', 'Gourmand', 'Green', 'Iris', 'Jasmine', 'Lily', 'Mossy', 'Musk', 'Orange', 'Rich', 'Richer', 'Rose', 'Soft', 'Spicy', 'Tuberose', 'Valley', 'Violet', 'Water', 'White', 'Woods', 'Woody']


In [3]:

# -------------------------------
# 4) 예측 실행
# -------------------------------
example_text = "달달한 향 추천좀"
print("\n[Example Prediction]")
print(predict_multilingual(example_text, topk=3, thresholds=thresholds))


[Example Prediction]
['Amber', 'Fresher']


In [ ]:
def recommend_perfume_simple(
    user_text: str,
    topk_labels: int = 3,
    top_n_perfumes: int = 5,
    use_thresholds: bool = True,
    model_pkl_path: str = "./models.pkl",
    perfume_json_path: str = "perfumes.json",
    model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    max_len: int = 256,
    global_threshold: float = 0.4,
    min_labels: int = 0,
    # --- NEW: abstain gates ---
    domain_gate: bool = True,
    domain_tau: float = 0.17,      # 코사인 유사도 하한 (0.18~0.25 권장)
    max_proba_min: float = 0.60,   # 최대 확률 하한 (0.55~0.65 권장)
    per_label_floor: float = 0.40  # per-label threshold의 최저 바닥값
):
    import json, numpy as np, torch, joblib
    from transformers import AutoTokenizer, AutoModel
    from rank_bm25 import BM25Okapi

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 1) Load ML bundle
    data = joblib.load(model_pkl_path)
    clf = data["classifier"]
    mlb = data["mlb"]
    per_label_thresholds = (data.get("thresholds", {}) or {})  # dict: label -> float

    # 2) Encoder
    tok = AutoTokenizer.from_pretrained(model_name)
    enc_model = AutoModel.from_pretrained(model_name).to(device)
    enc_model.eval()

    # 3) Load perfumes
    with open(perfume_json_path, "r", encoding="utf-8") as f:
        perfumes = json.load(f)
        if not isinstance(perfumes, list):
            raise ValueError("perfumes.json must contain a list of perfume objects")

    # 4) BM25 index
    def doc_of(p):
        fr = p.get("fragrances")
        if isinstance(fr, list):
            text = " ".join(map(str, fr))
        elif isinstance(fr, str):
            text = fr
        else:
            text = " ".join(
                str(x) for x in [
                    p.get("description", ""),
                    p.get("main_accords", ""),
                    p.get("name_perfume") or p.get("name", ""),
                    p.get("brand", ""),
                ] if x
            )
        return (text or "unknown").lower()

    tokenized_corpus = [doc_of(p).split() for p in perfumes]
    bm25 = BM25Okapi(tokenized_corpus)

    # 5) Encode text -> vector
    batch = tok([user_text], padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(device)
    with torch.no_grad():
        out = enc_model(**batch)
        emb = out.last_hidden_state.mean(dim=1).cpu().numpy()  # (1, d)

    # --- Gate A: Domain similarity (쿼리 vs 도메인 대표문장) ---
    if domain_gate:
        ref_text = (
            "perfume fragrance scent cologne eau de parfum eau de toilette "
            "citrus woody floral musk amber vanilla powdery aquatic green spicy "
            "fresh sweet leather tobacco rose jasmine sandalwood patchouli vetiver bergamot lavender"
        )
        ref_batch = tok([ref_text], padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(device)
        with torch.no_grad():
            ref_out = enc_model(**ref_batch)
            ref_emb = ref_out.last_hidden_state.mean(dim=1).cpu().numpy()
        # cosine similarity
        def _cos(a, b):
            a = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-12)
            b = b / (np.linalg.norm(b, axis=1, keepdims=True) + 1e-12)
            return float((a @ b.T)[0, 0])
        cos_sim = _cos(emb, ref_emb)
        if cos_sim < float(domain_tau):
            return {
                "user_input": user_text,
                "predicted_labels": [],
                "recommendations": [],
                "meta": {
                    "rejected": True,
                    "reason": "DOMAIN_SIM_LOW",
                    "cos_sim": cos_sim,
                    "domain_tau": float(domain_tau)
                },
            }

    # 6) Predict probs
    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(emb)[0]
    elif hasattr(clf, "decision_function"):
        logits = np.asarray(clf.decision_function(emb)[0], dtype=float)
        proba = 1.0 / (1.0 + np.exp(-logits))
    else:
        proba = np.asarray(clf.predict(emb)[0], dtype=float)

    # --- Gate B: Max probability lower bound ---
    if float(np.max(proba)) < float(max_proba_min):
        return {
            "user_input": user_text,
            "predicted_labels": [],
            "recommendations": [],
            "meta": {
                "rejected": True,
                "reason": "LOW_CONFIDENCE_MAXP",
                "max_proba": float(np.max(proba)),
                "max_proba_min": float(max_proba_min)
            },
        }

    classes = list(mlb.classes_)

    # === Threshold 기반 라벨 선택 (per-label 바닥값 적용) ===
    if use_thresholds and per_label_thresholds:
        th_vec = np.array([max(float(per_label_thresholds.get(c, global_threshold)), float(per_label_floor))
                           for c in classes], dtype=float)
    else:
        th_vec = np.full_like(proba, fill_value=float(global_threshold), dtype=float)

    picked_idx = [i for i, p in enumerate(proba) if p >= th_vec[i]]

    # 최소 라벨 보장
    if len(picked_idx) < int(min_labels):
        order = np.argsort(-proba)
        need = max(int(min_labels), min(len(order), int(topk_labels) if topk_labels else len(order)))
        picked_idx = order[:need].tolist()

    # 상한 컷
    if topk_labels and len(picked_idx) > int(topk_labels):
        picked_idx = sorted(picked_idx, key=lambda i: proba[i], reverse=True)[:int(topk_labels)]

    labels = [classes[i] for i in picked_idx]

    # --- Gate C: No labels → reject (fallback 금지) ---
    if len(labels) == 0:
        return {
            "user_input": user_text,
            "predicted_labels": [],
            "recommendations": [],
            "meta": {
                "rejected": True,
                "reason": "EMPTY_LABELS_LOW_CONFIDENCE"
            },
        }

    # 7) Retrieve with BM25 (labels만 사용)
    query_tokens = " ".join(labels).lower().split()
    scores = bm25.get_scores(query_tokens)
    top_idx = np.argsort(scores)[-top_n_perfumes:][::-1]

    def _safe(d, *keys, default="N/A"):
        for k in keys:
            if k in d and d[k] not in (None, ""):
                return d[k]
        return default

    recs = []
    for rnk, idx in enumerate(top_idx, 1):
        p = perfumes[int(idx)]
        fr = p.get("fragrances")
        fr_text = ", ".join(map(str, fr)) if isinstance(fr, list) else (fr if isinstance(fr, str) else _safe(p, "main_accords"))
        recs.append({
            "rank": int(rnk),
            "index": int(idx),
            "score": float(scores[int(idx)]),
            "brand": _safe(p, "brand"),
            "name": _safe(p, "name_perfume", "name"),
            "fragrances": fr_text,
            "perfume_data": p,
        })

    return {
        "user_input": user_text,
        "predicted_labels": labels,
        "recommendations": recs,
        "meta": {
                "model_name": model_name,
                "device": device,
                "max_len": int(max_len),
                "db_size": int(len(perfumes)),
                "threshold_mode": "per-label" if (use_thresholds and per_label_thresholds) else "global",
                "global_threshold": float(global_threshold),
                "min_labels": int(min_labels),
                "max_labels": int(topk_labels) if topk_labels else None,
                "rejected": False
        },
    }


In [24]:

# -------------------------------
# 4) 예측 실행
# -------------------------------
example_text = "LELU 추천좀"
recommend_perfume_simple("example_text")

c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [16:17:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:359: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [16:17:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:384: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [16:17:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  setstate(state)
c:\Users\Playdata2

{'user_input': 'example_text',
 'predicted_labels': ['Fresher'],
 'recommendations': [{'rank': 1,
   'index': 12014,
   'score': 1.6456789611296945,
   'brand': 'Dolce & Gabbana',
   'name': 'Light Blue Sun Pour Homme 2019',
   'fragrances': 'Water Fresher',
   'perfume_data': {'brand': 'Dolce & Gabbana',
    'name_perfume': 'Light Blue Sun Pour Homme 2019',
    'family': 'AROMATIC FOUGERE',
    'subfamily': 'WATERY',
    'fragrances': 'Water Fresher',
    'ingredients': ['Bergamot',
     'Oakmoss',
     'Rosemary',
     'Ozonic Notes',
     'Cedarwood',
     'Ginger',
     'Grapefruit',
     'Musk',
     'Vanilla',
     'Vetiver'],
    'origin': 'Italy',
    'gender': 'Male',
    'years': '2019',
    'description': 'On the enchanting island of Capri, two hearts race with the intoxicating magic of summer love. Hand in hand, skin warmed by the dazzling Mediterranean sun rays, their golden auras shimmer in the sun as they snatch playful kisses at every turn.',
    'image_name': 'eqkqvkfl

In [30]:
import json
import numpy as np
import torch
import joblib
from transformers import AutoTokenizer, AutoModel
from rank_bm25 import BM25Okapi
import pinecone
from typing import List, Dict, Any, Optional
import os
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class PineconeKeywordVectorDB:
    """파인콘을 사용한 키워드 벡터 검색 시스템"""
    
    def __init__(
        self, 
        api_key: str,
        environment: str = "us-west1-gcp",
        index_name: str = "perfume-keywords",
        dimension: int = 384,
        metric: str = "cosine"
    ):
        self.api_key = api_key
        self.environment = environment
        self.index_name = index_name
        self.dimension = dimension
        self.metric = metric
        self.index = None
        
        # 파인콘 초기화
        pinecone.init(api_key=api_key, environment=environment)
        
    def create_or_connect_index(self):
        """인덱스 생성 또는 연결"""
        try:
            if self.index_name not in pinecone.list_indexes():
                pinecone.create_index(
                    name=self.index_name,
                    dimension=self.dimension,
                    metric=self.metric
                )
                logger.info(f"Created new Pinecone index: {self.index_name}")
            
            self.index = pinecone.Index(self.index_name)
            logger.info(f"Connected to Pinecone index: {self.index_name}")
            
        except Exception as e:
            logger.error(f"Error connecting to Pinecone: {e}")
            raise
    
    def build_keyword_vectors(self, perfumes: List[Dict], model_name: str, max_len: int = 256):
        """향수 데이터로부터 키워드 벡터 생성 및 업로드"""
        if not self.index:
            raise RuntimeError("Index not connected. Call create_or_connect_index() first.")
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModel.from_pretrained(model_name).to(device)
        model.eval()
        
        vectors_to_upsert = []
        
        for i, perfume in enumerate(perfumes):
            # 키워드 텍스트 생성
            keywords = self._extract_keywords(perfume)
            keyword_text = " ".join(keywords)
            
            # 벡터 임베딩 생성
            batch = tokenizer([keyword_text], padding=True, truncation=True, 
                            max_length=max_len, return_tensors="pt").to(device)
            
            with torch.no_grad():
                output = model(**batch)
                vector = output.last_hidden_state.mean(dim=1).cpu().numpy()[0].tolist()
            
            # 메타데이터 준비
            metadata = {
                "perfume_index": i,
                "brand": perfume.get("brand", ""),
                "name": perfume.get("name_perfume") or perfume.get("name", ""),
                "keywords": keywords[:10],  # 상위 10개 키워드만 저장
                "main_accords": perfume.get("main_accords", ""),
                "description": perfume.get("description", "")[:500]  # 설명은 500자로 제한
            }
            
            vectors_to_upsert.append({
                "id": f"perfume_{i}",
                "values": vector,
                "metadata": metadata
            })
            
            # 배치 업로드 (100개씩)
            if len(vectors_to_upsert) >= 100:
                self.index.upsert(vectors_to_upsert)
                logger.info(f"Uploaded batch of {len(vectors_to_upsert)} vectors")
                vectors_to_upsert = []
        
        # 남은 벡터들 업로드
        if vectors_to_upsert:
            self.index.upsert(vectors_to_upsert)
            logger.info(f"Uploaded final batch of {len(vectors_to_upsert)} vectors")
    
    def _extract_keywords(self, perfume: Dict) -> List[str]:
        """향수 데이터에서 키워드 추출"""
        keywords = []
        
        # 향료 정보
        fragrances = perfume.get("fragrances", [])
        if isinstance(fragrances, list):
            keywords.extend([str(f).lower().strip() for f in fragrances])
        elif isinstance(fragrances, str):
            keywords.extend([f.strip().lower() for f in fragrances.split(",") if f.strip()])
        
        # 메인 어코드
        main_accords = perfume.get("main_accords", "")
        if main_accords:
            keywords.extend([a.strip().lower() for a in str(main_accords).split(",") if a.strip()])
        
        # 브랜드와 이름
        brand = perfume.get("brand", "")
        name = perfume.get("name_perfume") or perfume.get("name", "")
        if brand:
            keywords.append(brand.lower().strip())
        if name:
            keywords.extend([w.lower().strip() for w in str(name).split() if len(w) > 2])
        
        # 설명에서 키워드 추출 (간단한 방식)
        description = perfume.get("description", "")
        if description:
            # 향수 관련 키워드들만 추출
            perfume_terms = [
                "fresh", "sweet", "woody", "floral", "citrus", "musky", "spicy",
                "vanilla", "amber", "powdery", "aquatic", "green", "leather",
                "tobacco", "rose", "jasmine", "sandalwood", "patchouli", "vetiver",
                "bergamot", "lavender", "fruity", "oriental", "gourmand"
            ]
            desc_words = str(description).lower().split()
            keywords.extend([w for w in desc_words if w in perfume_terms])
        
        # 중복 제거 및 빈 문자열 제거
        keywords = list(set([k for k in keywords if k and len(k) > 1]))
        
        return keywords
    
    def search_similar_perfumes(
        self, 
        query_vector: np.ndarray, 
        top_k: int = 5,
        min_score: float = 0.3
    ) -> List[Dict]:
        """벡터 유사도로 향수 검색"""
        if not self.index:
            raise RuntimeError("Index not connected")
        
        # 쿼리 실행
        results = self.index.query(
            vector=query_vector.tolist(),
            top_k=top_k,
            include_metadata=True
        )
        
        recommendations = []
        for i, match in enumerate(results.matches):
            if match.score >= min_score:
                metadata = match.metadata
                recommendations.append({
                    "rank": i + 1,
                    "index": metadata["perfume_index"],
                    "score": float(match.score),
                    "brand": metadata.get("brand", "N/A"),
                    "name": metadata.get("name", "N/A"),
                    "fragrances": ", ".join(metadata.get("keywords", [])),
                    "similarity_score": float(match.score),
                    "search_method": "pinecone_vector_similarity"
                })
        
        return recommendations


def recommend_perfume_with_fallback(
    user_text: str,
    topk_labels: int = 3,
    top_n_perfumes: int = 5,
    use_thresholds: bool = True,
    model_pkl_path: str = "./models.pkl",
    perfume_json_path: str = "perfumes.json",
    model_name: str = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    max_len: int = 256,
    global_threshold: float = 0.4,
    min_labels: int = 0,
    # 기존 abstain gates
    domain_gate: bool = True,
    domain_tau: float = 0.17,
    max_proba_min: float = 0.60,
    per_label_floor: float = 0.40,
    # 파인콘 설정
    pinecone_api_key: Optional[str] = None,
    pinecone_environment: str = "us-west1-gcp",
    use_pinecone_fallback: bool = True,
    pinecone_min_score: float = 0.3
):
    """향수 추천 함수 - 파인콘 키워드 벡터DB 대체 포함"""
    
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 1) Load ML bundle
    data = joblib.load(model_pkl_path)
    clf = data["classifier"]
    mlb = data["mlb"]
    per_label_thresholds = (data.get("thresholds", {}) or {})

    # 2) Encoder
    tok = AutoTokenizer.from_pretrained(model_name)
    enc_model = AutoModel.from_pretrained(model_name).to(device)
    enc_model.eval()

    # 3) Load perfumes
    with open(perfume_json_path, "r", encoding="utf-8") as f:
        perfumes = json.load(f)
        if not isinstance(perfumes, list):
            raise ValueError("perfumes.json must contain a list of perfume objects")

    # 4) BM25 index
    def doc_of(p):
        fr = p.get("fragrances")
        if isinstance(fr, list):
            text = " ".join(map(str, fr))
        elif isinstance(fr, str):
            text = fr
        else:
            text = " ".join(
                str(x) for x in [
                    p.get("description", ""),
                    p.get("main_accords", ""),
                    p.get("name_perfume") or p.get("name", ""),
                    p.get("brand", ""),
                ] if x
            )
        return (text or "unknown").lower()

    tokenized_corpus = [doc_of(p).split() for p in perfumes]
    bm25 = BM25Okapi(tokenized_corpus)

    # 5) Encode text -> vector
    batch = tok([user_text], padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(device)
    with torch.no_grad():
        out = enc_model(**batch)
        emb = out.last_hidden_state.mean(dim=1).cpu().numpy()

    # Gate A: Domain similarity
    if domain_gate:
        ref_text = (
            "perfume fragrance scent cologne eau de parfum eau de toilette "
            "citrus woody floral musk amber vanilla powdery aquatic green spicy "
            "fresh sweet leather tobacco rose jasmine sandalwood patchouli vetiver bergamot lavender"
        )
        ref_batch = tok([ref_text], padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(device)
        with torch.no_grad():
            ref_out = enc_model(**ref_batch)
            ref_emb = ref_out.last_hidden_state.mean(dim=1).cpu().numpy()

        def _cos(a, b):
            a = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-12)
            b = b / (np.linalg.norm(b, axis=1, keepdims=True) + 1e-12)
            return float((a @ b.T)[0, 0])

        cos_sim = _cos(emb, ref_emb)
        if cos_sim < float(domain_tau):
            return {
                "user_input": user_text,
                "predicted_labels": [],
                "recommendations": [],
                "meta": {
                    "rejected": True,
                    "reason": "DOMAIN_SIM_LOW",
                    "cos_sim": cos_sim,
                    "domain_tau": float(domain_tau)
                },
            }

    # 6) Predict probs
    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(emb)[0]
    elif hasattr(clf, "decision_function"):
        logits = np.asarray(clf.decision_function(emb)[0], dtype=float)
        proba = 1.0 / (1.0 + np.exp(-logits))
    else:
        proba = np.asarray(clf.predict(emb)[0], dtype=float)

    # Gate B: Max probability lower bound
    if float(np.max(proba)) < float(max_proba_min):
        return {
            "user_input": user_text,
            "predicted_labels": [],
            "recommendations": [],
            "meta": {
                "rejected": True,
                "reason": "LOW_CONFIDENCE_MAXP",
                "max_proba": float(np.max(proba)),
                "max_proba_min": float(max_proba_min)
            },
        }

    classes = list(mlb.classes_)

    # 임계값 기반 라벨 선택
    if use_thresholds and per_label_thresholds:
        th_vec = np.array([max(float(per_label_thresholds.get(c, global_threshold)), float(per_label_floor))
                           for c in classes], dtype=float)
    else:
        th_vec = np.full_like(proba, fill_value=float(global_threshold), dtype=float)

    picked_idx = [i for i, p in enumerate(proba) if p >= th_vec[i]]

    # 최소 라벨 보장
    if len(picked_idx) < int(min_labels):
        order = np.argsort(-proba)
        need = max(int(min_labels), min(len(order), int(topk_labels) if topk_labels else len(order)))
        picked_idx = order[:need].tolist()

    # 상한 컷
    if topk_labels and len(picked_idx) > int(topk_labels):
        picked_idx = sorted(picked_idx, key=lambda i: proba[i], reverse=True)[:int(topk_labels)]

    labels = [classes[i] for i in picked_idx]

    # === 여기가 핵심: 빈 라벨일 때 파인콘 대체 시스템 ===
    if len(labels) == 0 and use_pinecone_fallback and pinecone_api_key:
        logger.info("Empty labels detected. Falling back to Pinecone keyword vector search.")
        
        try:
            # 파인콘 벡터DB 초기화
            pinecone_db = PineconeKeywordVectorDB(
                api_key=pinecone_api_key,
                environment=pinecone_environment
            )
            pinecone_db.create_or_connect_index()
            
            # 파인콘에서 유사도 검색
            recommendations = pinecone_db.search_similar_perfumes(
                query_vector=emb[0],
                top_k=top_n_perfumes,
                min_score=pinecone_min_score
            )
            
            return {
                "user_input": user_text,
                "predicted_labels": [],
                "recommendations": recommendations,
                "meta": {
                    "model_name": model_name,
                    "device": device,
                    "rejected": False,
                    "fallback_used": "pinecone_vector_similarity",
                    "pinecone_min_score": pinecone_min_score,
                    "original_reason": "EMPTY_LABELS_LOW_CONFIDENCE"
                },
            }
            
        except Exception as e:
            logger.error(f"Pinecone fallback failed: {e}")
            # 파인콘 실패 시 원래대로 빈 결과 반환
            return {
                "user_input": user_text,
                "predicted_labels": [],
                "recommendations": [],
                "meta": {
                    "rejected": True,
                    "reason": "EMPTY_LABELS_LOW_CONFIDENCE",
                    "pinecone_error": str(e)
                },
            }
    
    elif len(labels) == 0:
        # 파인콘 사용하지 않거나 API 키가 없는 경우
        return {
            "user_input": user_text,
            "predicted_labels": [],
            "recommendations": [],
            "meta": {
                "rejected": True,
                "reason": "EMPTY_LABELS_LOW_CONFIDENCE"
            },
        }

    # 7) 기존 BM25 검색 (라벨이 있는 경우)
    query_tokens = " ".join(labels).lower().split()
    scores = bm25.get_scores(query_tokens)
    top_idx = np.argsort(scores)[-top_n_perfumes:][::-1]

    def _safe(d, *keys, default="N/A"):
        for k in keys:
            if k in d and d[k] not in (None, ""):
                return d[k]
        return default

    recs = []
    for rnk, idx in enumerate(top_idx, 1):
        p = perfumes[int(idx)]
        fr = p.get("fragrances")
        fr_text = ", ".join(map(str, fr)) if isinstance(fr, list) else (fr if isinstance(fr, str) else _safe(p, "main_accords"))
        recs.append({
            "rank": int(rnk),
            "index": int(idx),
            "score": float(scores[int(idx)]),
            "brand": _safe(p, "brand"),
            "name": _safe(p, "name_perfume", "name"),
            "fragrances": fr_text,
            "search_method": "bm25_with_ml_labels"
        })

    return {
        "user_input": user_text,
        "predicted_labels": labels,
        "recommendations": recs,
        "meta": {
            "model_name": model_name,
            "device": device,
            "max_len": int(max_len),
            "db_size": int(len(perfumes)),
            "threshold_mode": "per-label" if (use_thresholds and per_label_thresholds) else "global",
            "global_threshold": float(global_threshold),
            "min_labels": int(min_labels),
            "max_labels": int(topk_labels) if topk_labels else None,
            "rejected": False
        },
    }


# === 테스트 코드 ===
def generate_test_queries():
    """테스트 쿼리 생성"""
    test_queries = [
        # 향수 도메인 관련 (정상적으로 처리되어야 함)
        "I want a fresh and citrusy perfume for summer",
        "Looking for something woody and masculine",
        "Sweet vanilla scent for evening wear",
        "Floral fragrance for romantic dates",
        "Clean and powdery scent for office",
        
        # 경계선 케이스 (낮은 확신도로 빈 라벨 가능성)
        "something light and airy",
        "I need good smell",
        "nice scent please",
        "perfume for special occasion",
        "fragrance recommendation",
        
        # 도메인 밖 쿼리 (도메인 게이트에서 거부되어야 함)
        "I want to cook pasta today",
        "How to fix my car engine",
        "Best programming language to learn",
        "Weather forecast for tomorrow",
        "Math homework help needed"
    ]
    return test_queries

def run_comprehensive_test(
    model_pkl_path: str = "./models.pkl",
    perfume_json_path: str = "perfumes.json", 
    pinecone_api_key: str = None,
    output_file: str = "keyword_test.txt"
):
    """종합 테스트 실행 및 결과 저장"""
    
    test_queries = generate_test_queries()
    results = []
    
    print(f"Running comprehensive test with {len(test_queries)} queries...")
    print("=" * 60)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n[Test {i}/{len(test_queries)}] Query: '{query}'")
        
        try:
            result = recommend_perfume_with_fallback(
                user_text=query,
                model_pkl_path=model_pkl_path,
                perfume_json_path=perfume_json_path,
                pinecone_api_key=pinecone_api_key,
                use_pinecone_fallback=True if pinecone_api_key else False,
                # 테스트를 위해 임계값을 높여서 빈 라벨 유도
                max_proba_min=0.70,
                global_threshold=0.50
            )
            
            # 결과 분석
            is_rejected = result["meta"].get("rejected", False)
            fallback_used = result["meta"].get("fallback_used")
            num_recommendations = len(result["recommendations"])
            num_labels = len(result["predicted_labels"])
            
            status = "REJECTED" if is_rejected else "SUCCESS"
            if fallback_used:
                status += f" (Fallback: {fallback_used})"
            
            print(f"  Status: {status}")
            print(f"  Labels: {num_labels} ({result['predicted_labels']})")
            print(f"  Recommendations: {num_recommendations}")
            
            if result["recommendations"]:
                top_rec = result["recommendations"][0]
                print(f"  Top recommendation: {top_rec['brand']} - {top_rec['name']}")
            
            # 결과 저장
            results.append({
                "query": query,
                "result": result,
                "test_summary": {
                    "status": status,
                    "num_labels": num_labels,
                    "num_recommendations": num_recommendations,
                    "is_rejected": is_rejected,
                    "fallback_used": fallback_used
                }
            })
            
        except Exception as e:
            print(f"  ERROR: {e}")
            results.append({
                "query": query,
                "error": str(e),
                "test_summary": {
                    "status": "ERROR",
                    "error": str(e)
                }
            })
    
    # 결과를 파일에 저장
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("PERFUME RECOMMENDATION SYSTEM - COMPREHENSIVE TEST RESULTS\n")
        f.write("=" * 80 + "\n\n")
        f.write(f"Total queries tested: {len(test_queries)}\n")
        f.write(f"Timestamp: {pd.Timestamp.now()}\n\n")
        
        # 통계 요약
        success_count = sum(1 for r in results if r.get("test_summary", {}).get("status", "").startswith("SUCCESS"))
        rejected_count = sum(1 for r in results if r.get("test_summary", {}).get("is_rejected", False))
        fallback_count = sum(1 for r in results if r.get("test_summary", {}).get("fallback_used"))
        error_count = sum(1 for r in results if "error" in r)
        
        f.write("SUMMARY STATISTICS:\n")
        f.write("-" * 40 + "\n")
        f.write(f"Successful recommendations: {success_count}\n")
        f.write(f"Rejected queries: {rejected_count}\n")
        f.write(f"Fallback used: {fallback_count}\n")
        f.write(f"Errors: {error_count}\n\n")
        
        # 상세 결과
        f.write("DETAILED RESULTS:\n")
        f.write("-" * 40 + "\n\n")
        
        for i, result_data in enumerate(results, 1):
            f.write(f"[TEST {i}] Query: '{result_data['query']}'\n")
            
            if "error" in result_data:
                f.write(f"ERROR: {result_data['error']}\n\n")
                continue
            
            result = result_data["result"]
            summary = result_data["test_summary"]
            
            f.write(f"Status: {summary['status']}\n")
            f.write(f"Predicted Labels ({summary['num_labels']}): {result['predicted_labels']}\n")
            f.write(f"Recommendations ({summary['num_recommendations']}):\n")
            
            for rec in result["recommendations"][:3]:  # 상위 3개만 표시
                f.write(f"  {rec['rank']}. {rec['brand']} - {rec['name']} (Score: {rec['score']:.3f})\n")
            
            f.write(f"Meta: {json.dumps(result['meta'], indent=2, ensure_ascii=False)}\n")
            f.write("\n" + "="*60 + "\n\n")
    
    print(f"\n\nTest completed! Results saved to {output_file}")
    print(f"Success: {success_count}, Rejected: {rejected_count}, Fallback: {fallback_count}, Errors: {error_count}")
    
    return results

# 사용 예시
if __name__ == "__main__":
    # 파인콘 API 키 설정 (환경변수에서 가져오거나 직접 입력)
    PINECONE_API_KEY = os.getenv("PINECONE_API_KEY", "your-pinecone-api-key-here")
    
    # 테스트 실행
    try:
        test_results = run_comprehensive_test(
            model_pkl_path="./models.pkl",
            perfume_json_path="perfumes.json",
            pinecone_api_key=PINECONE_API_KEY if PINECONE_API_KEY != "your-pinecone-api-key-here" else None,
            output_file="keyword_test.txt"
        )
        
        # 파인콘 인덱스 초기 구축 (한 번만 실행)
        if PINECONE_API_KEY and PINECONE_API_KEY != "your-pinecone-api-key-here":
            print("\nBuilding Pinecone keyword vector index...")
            with open("perfumes.json", "r", encoding="utf-8") as f:
                perfumes = json.load(f)
            
            pinecone_db = PineconeKeywordVectorDB(api_key=PINECONE_API_KEY)
            pinecone_db.create_or_connect_index()
            pinecone_db.build_keyword_vectors(
                perfumes=perfumes,
                model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
            )
            print("Pinecone index building completed!")
        
    except Exception as e:
        print(f"Test execution failed: {e}")

Running comprehensive test with 15 queries...

[Test 1/15] Query: 'I want a fresh and citrusy perfume for summer'


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:30:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:359: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:30:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:384: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:30:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  setstate(state)
c:\Users\Playdata2

  Status: SUCCESS
  Labels: 2 (['Citrus', 'Fresher'])
  Recommendations: 5
  Top recommendation: Molton Brown - Sunlit Clementine & Vetiver Eau De Parfum

[Test 2/15] Query: 'Looking for something woody and masculine'


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:30:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:359: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:30:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:384: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:30:12] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  setstate(state)
c:\Users\Playdata2

  Status: SUCCESS
  Labels: 3 (['Aromatic', 'Fougère', 'Fresher'])
  Recommendations: 5
  Top recommendation: Avon - Full Speed Max Turbo

[Test 3/15] Query: 'Sweet vanilla scent for evening wear'


...

  Status: REJECTED
  Labels: 0 ([])
  Recommendations: 0

[Test 14/15] Query: 'Weather forecast for tomorrow'


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:31:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:359: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:31:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:384: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:31:06] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  setstate(state)
c:\Users\Playdata2

  Status: REJECTED
  Labels: 0 ([])
  Recommendations: 0

[Test 15/15] Query: 'Math homework help needed'


c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:31:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:359: 
  Loading from a raw memory buffer (like pickle in Python, RDS in R) on a CPU-only
  machine. Consider using `save_model/load_model` instead. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.  Changing `tree_method` to `hist`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:31:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\gbm\gbtree.cc:384: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  setstate(state)
c:\Users\Playdata2\miniconda3\envs\final-env\Lib\pickle.py:1760: UserWarning: [17:31:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\context.cc:49: No visible GPU is found, setting device to CPU.
  setstate(state)
c:\Users\Playdata2

  Status: REJECTED
  Labels: 0 ([])
  Recommendations: 0


Test completed! Results saved to keyword_test.txt
Success: 6, Rejected: 9, Fallback: 0, Errors: 0
